In [1]:
import av
import numpy as np

from transformers import Trainer, TrainingArguments, Seq2SeqTrainingArguments, DataCollatorForLanguageModeling
from transformers import AutoProcessor, BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, DataCollatorWithPadding 
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

import torch
from huggingface_hub import  hf_hub_download
import av
import torch
import numpy as np
from transformers import LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor, BitsAndBytesConfig
from huggingface_hub import hf_hub_download
import numpy as np
import evaluate
from activity_dataset import get_dataset_splits



MODEL_ID = "llava-hf/LLaVA-NeXT-Video-7B-hf"
USE_LORA = False
USE_QLORA = True

c:\Users\Informatica_UNICAM\Desktop\csd\activity-recognition\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import os

# # Set the environment variable before importing torch
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load the model in half-precision
processor = LlavaNextVideoProcessor.from_pretrained(MODEL_ID)
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map='auto',
)
processor.tokenizer.padding_side = "left"

Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.25s/it]


In [4]:
metric = evaluate.load("accuracy")

In [32]:
from torch.nn.utils.rnn import pad_sequence
from activity_dataset import num_lab

MAX_LENGTH = 256

def collate_fn(batch):
    # Extract video tensors and labels
    video_tensors = [video for video, _ in batch]
    labels = [label for _, label in batch]

    # Pad video tensors to have the same length
    video_tensors = pad_sequence(video_tensors, batch_first=True)  # Shape: (batch_size, seq_len, ...)
    
    print(labels)
    print(video_tensors.shape)

    # Prepare processor outputs
    outputs = []
    for video, label in zip(video_tensors, labels):
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Identify the activity of the robots in this video"},
                    {"type": "video"},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": num_lab[label]},
                ],
            },
        ]

        # Apply the processor's chat template
        prompt = processor.apply_chat_template(conversation, add_generation_prompt=False)

        # Process the inputs
        processed = processor(
            text=prompt,
            videos=video,
            truncation=True,
            max_length=MAX_LENGTH,
            return_tensors="pt"
        )
        # processed["labels"] = labels
        # processed["pixel_values_videos"] = torch.cat([feat['pixel_values_videos'] for feat in processed], dim=0)

        
        outputs.append(processed)

    
    # Combine processor outputs into a batch
    batch_outputs = {}
    for key in outputs[0]:  # Iterate over keys like 'input_ids', 'attention_mask', etc.
        batch_outputs[key] = torch.cat([output[key] for output in outputs], dim=0)
    # print(batch_outputs)

    # Add padded video tensors to the batch
    batch_outputs["pixel_values"] = video_tensors
    batch_outputs["labels"] = torch.tensor(labels)  # Convert labels to tensor
    batch_outputs["image_sizes"] = torch.tensor([(336, 336)] * video_tensors.size(0))  # Add image_sizes to the batch

    for k, v in batch_outputs.items():
        print(f"{k=}, {v.shape}")
    # print(batch_outputs)
    return batch_outputs

In [33]:
train_dataset, test_dataset = get_dataset_splits()
train_dataset

In [34]:
class LlavaNextVideoDataCollatorWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        # padded_inputs = self.processor.tokenizer.pad(
        #     {
        #         "input_ids": [feat['input_ids'][0] for feat in features], # each element is one batch only so we slice [0]
        #         "attention_mask": [feat['attention_mask'][0] for feat in features],
        #     },
        #     padding=True,
        #     return_tensors="pt",
        # )

        # labels = padded_inputs["input_ids"].clone()
        # labels[labels == self.processor.tokenizer.pad_token_id] = -100
        # padded_inputs["labels"] = labels
        # padded_inputs["pixel_values_videos"] = torch.cat([feat['pixel_values_videos'] for feat in features], dim=0)
        print(features)

        return {"pixel_values_videos": features[0], "label" : features[1], "len":features[2]}

In [35]:
if USE_QLORA or USE_LORA:
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
    model = LlavaNextVideoForConditionalGeneration.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
        device_map="auto",
    )

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.35s/it]


In [36]:
def find_all_linear_names(model):
    cls = torch.nn.Linear
    lora_module_names = set()
    multimodal_keywords = ['multi_modal_projector', 'vision_model']
    for name, module in model.named_modules():
        if any(mm_keyword in name for mm_keyword in multimodal_keywords):
            continue
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)


lora_config = LoraConfig(
    r=8,
    lora_alpha=8,
    lora_dropout=0.1,
    target_modules=find_all_linear_names(model),
    init_lora_weights="gaussian",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [37]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [38]:
OUTPUT_DIR = "output_llava"
BATCH_SIZE = 1
REPO_ID = "cams01/LLaVa-robot-activity-recognition"

args = TrainingArguments(


    # args related to training
    output_dir = OUTPUT_DIR,
    eval_strategy = 'steps',
    eval_steps=20,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    gradient_accumulation_steps = 8,
    learning_rate = 2e-05,
    max_steps = 100, # adjust this depending on your dataset size
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.1,

    # args related to eval/save
    logging_steps = 20,
    save_strategy = 'steps',
    save_steps=20,
    save_total_limit = 1,
    fp16 = True, # we have the model train and eval with fp16 precision
    fp16_full_eval = True,
    optim = 'adamw_bnb_8bit', # adam in lower-bits to save memory, consider changing to 'adamw_torch' if model is not converging
    # report_to = "wandb", # install wand to use this
    # hub_model_id = REPO_ID,
    # push_to_hub = True, # wel'll push the model to hub after each epoch

    # model that was wrapped for QLORA training with peft will not have arguments listed in its signature
    # so we need to pass lable names explicitly to calculate val loss
    label_names=["labels"],
    # dataloader_num_workers=4, # let's get more workers since iterating on video datasets might be slower in general
)

In [39]:
trainer = Trainer(
    model = model,
    processing_class = processor,
    #data_collator = DataCollatorWithPadding(processor),
    # data_collator = LlavaNextVideoDataCollatorWithPadding(processor=processor),
    data_collator = collate_fn,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    args=args,
)

In [40]:
trainer.train()

[3]
torch.Size([1, 205, 3, 336, 336])
k='input_ids', torch.Size([1, 256])
k='attention_mask', torch.Size([1, 256])
k='pixel_values_videos', torch.Size([1, 205, 3, 336, 336])
k='pixel_values', torch.Size([1, 205, 3, 336, 336])
k='labels', torch.Size([1])
k='image_sizes', torch.Size([1, 2])
[3]
torch.Size([1, 267, 3, 336, 336])
k='input_ids', torch.Size([1, 256])
k='attention_mask', torch.Size([1, 256])
k='pixel_values_videos', torch.Size([1, 267, 3, 336, 336])
k='pixel_values', torch.Size([1, 267, 3, 336, 336])
k='labels', torch.Size([1])
k='image_sizes', torch.Size([1, 2])
[7]
torch.Size([1, 176, 3, 336, 336])
k='input_ids', torch.Size([1, 256])
k='attention_mask', torch.Size([1, 256])
k='pixel_values_videos', torch.Size([1, 176, 3, 336, 336])
k='pixel_values', torch.Size([1, 176, 3, 336, 336])
k='labels', torch.Size([1])
k='image_sizes', torch.Size([1, 2])
[5]
torch.Size([1, 100, 3, 336, 336])
k='input_ids', torch.Size([1, 256])
k='attention_mask', torch.Size([1, 256])
k='pixel_values

c:\Users\Informatica_UNICAM\Desktop\csd\activity-recognition\venv\lib\site-packages\torch\_dynamo\eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Informatica_UNICAM\Desktop\csd\activity-recognition\venv\lib\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


ValueError: Image features and image tokens do not match: tokens: 0, features 1176

In [ ]:
trainer.model.push_to_hub(REPO_ID) # let's push to hub the last ckpt